# Xarray Example of Graph Execution

## Graph Execution

To use a different backend for the execution, pass the required backend to the `Fluent` constructor. Currently supported backends include `array_api_compat`, which is the default, `xarray` and `jax`.

In [ ]:
import xarray as xr

ds = xr.load_dataset('rasm.nc')
print(ds)

In [ ]:
from cascade.fluent import Fluent, Payload
from cascade.backends.xarray import XArrayBackend

graph = (
    Fluent(backend=XArrayBackend)
    .source(xr.load_dataset, ("rasm.nc",))
    .map(Payload(lambda x: x.groupby("time.season")))
    .sum(dim="time")
    .expand("season", 4, 1, 0)
    .minimum("season")
    .graph()
)

In [ ]:
from cascade.executors.dask import DaskLocalExecutor

DaskLocalExecutor.execute(graph, memory_limit="5GB", n_workers=1, threads_per_worker=1)